In [1]:
import sys
import snowflake.connector as sf
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
ctx =sf.connect(
    user='mgupta1@chewy.com',
    autocommit = 0,
    authenticator= "externalbrowser",
    account='chewy.us-east-1'
    )

Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...
Going to open: https://chewy.okta.com/app/snowflake/exkmzig18yW58r3j00x7/sso/saml?SAMLRequest=lZJLc9owFIX%2Fikdd25J5JKDBZBwohE7aMoGUtjshCyNsS66ujKG%2FPjKPTLpIZrrzyOfoO7rnDu4ORe7thQGpVYTCgCBPKK4TqdIIPS8nfg95YJlKWK6ViNBRALobDoAVeUnjym7Vk%2FhTCbCeu0gBbX5EqDKKagYSqGKFAGo5XcRfH2krIJQBCGMdDl0sCUjH2lpbUozrug7qdqBNiluEEEz62KkaySf0BlF%2BzCiNtprr%2FGo5uDe9gwgx6TQIp3CE%2BcV4L9V5BB9R1mcR0Iflcu7Pvy%2BWyIuvrxtpBVUhzEKYveTi%2BenxHABcgtHD59WvoAJfMLB%2BGIDS9SZnmeC6KCvr7gzcF96IBOc6lW5Ss3GEykwmNh7djr%2Bku%2B00bvez1Wo6Tyd82m7z3aFO9gnfrbP79XbzsxNqjrwf115bTa8zgErMVNOmdUek1fHJjU86y7BLOz0a9oObVvc38sauTamYPTmvkflW1MdAZ5adorGyxK%2BpsThkxV%2BZhr3jqtsz7R0hh1sMoHFTFTpvCz3hzfD%2FZjDAb72XtfvmmpiN5zqX%2FOhNtCmYfb%2BoMAhPJzLxNycpFQWTeZwkRgC4wvJc1yMjmHXbbU0lEB6eqf%2Fu9%2FAF&RelayState=5744

In [3]:
sql_consider_cust = """ 

SELECT * FROM mrch_sandbox.mrch_ds_customer_segment_data
"""

cur= ctx.cursor()
cur.execute(sql_consider_cust)
data_consider = cur.fetch_pandas_all()
data_consider.shape

(7429352, 8)

In [6]:
data_consider["SW_PROMO_FLAG"].value_counts()

SW_PROMO_FLAG
NO_PROMO        7072562
SINGLE_PROMO     297780
MULTI_PROMO       59010
Name: count, dtype: int64

In [15]:
data_consider["SW_FLAG"] = 0
data_consider.loc[data_consider["SW_PROMO_FLAG"].isin(['SINGLE_PROMO' , 'MULTI_PROMO'])  , "SW_FLAG" ] = 1

data_consider["CONCAT"] = data_consider['AS_FLAG'].astype(str) + '_'  + data_consider['PET_FLAG'].astype(str) + '_'  + data_consider['MC2_FLAG'].astype(str) + '_'   + data_consider['SW_FLAG'].astype(str)

data_consider['COUNT'] = data_consider.groupby(['BUCKETS' , 'REGULAR_FLAG' , 'CONSISTENT_FLAG'])['CUSTOMER_ID'].transform('count')
data_consider["COMBO_COUNT"] = data_consider.groupby(['BUCKETS' , 'REGULAR_FLAG' , 'CONSISTENT_FLAG', 'CONCAT'])['CUSTOMER_ID'].transform('nunique')

data_consider["COMBO"] =  data_consider['REGULAR_FLAG'].astype(str) + '_'  + data_consider['CONSISTENT_FLAG'].astype(str) + '_'  + data_consider['BUCKETS'].astype(str)



In [16]:
data_consider.sort_values(by = "COMBO_COUNT")

,BUCKETS,REGULAR_FLAG,CONSISTENT_FLAG,CUSTOMER_ID,AS_FLAG,PET_FLAG,MC2_FLAG,SW_PROMO_FLAG,CONCAT,COUNT,COMBO_COUNT,COMBO,SW_FLAG
448014,2_ns_51_150,1,1,44576789,0,MULTI_PET,1,SINGLE_PROMO,0_MULTI_PET_1_1,55701,3,1_1_2_ns_51_150,1
3579664,1_ns_1_50,0,1,69756973,0,MULTI_PET,1,SINGLE_PROMO,0_MULTI_PET_1_1,586515,3,0_1_1_ns_1_50,1
1833121,4_ns_250_300,0,0,22634540,0,MULTI_PET,1,SINGLE_PROMO,0_MULTI_PET_1_1,79999,3,0_0_4_ns_250_300,1
801161,1_ns_1_50,0,1,91566196,0,MULTI_PET,1,SINGLE_PROMO,0_MULTI_PET_1_1,586515,3,0_1_1_ns_1_50,1
6329277,2_ns_51_150,1,1,891395791,0,MULTI_PET,1,SINGLE_PROMO,0_MULTI_PET_1_1,55701,3,1_1_2_ns_51_150,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6892976,2_ns_51_150,0,0,36837543,0,NO_PET,0,NO_PROMO,0_NO_PET_0_0,912965,282761,0_0_2_ns_51_150,0
6892975,2_ns_51_150,0,0,128971380,0,NO_PET,0,NO_PROMO,0_NO_PET_0_0,912965,282761,0_0_2_ns_51_150,0
5050344,2_ns_51_150,0,0,151341107,0,NO_PET,0,NO_PROMO,0_NO_PET_0_0,912965,282761,0_0_2_ns_51_150,0
620854,2_ns_51_150,0,0,23195565,0,NO_PET,0,NO_PROMO,0_NO_PET_0_0,912965,282761,0_0_2_ns_51_150,0


In [10]:
data_consider.groupby(['BUCKETS' , 'REGULAR_FLAG' , 'CONSISTENT_FLAG'])['CUSTOMER_ID'].count().sort_values()

BUCKETS         REGULAR_FLAG  CONSISTENT_FLAG
8_ns_1100_1200  0             1                    10301
9_ns_1200_1500  0             1                    18372
6_ns_500_700    0             0                    33595
2_ns_51_150     1             1                    55701
8_ns_1100_1200  1             1                    74039
4_ns_250_300    0             0                    79999
                1             1                   101591
5_ns_300_500    0             0                   102725
7_ns_700_1100   0             1                   114272
3_ns_150_250    1             1                   154484
9_ns_1200_1500  1             1                   162994
6_ns_500_700    0             1                   168227
4_ns_250_300    0             1                   186927
3_ns_150_250    0             0                   273872
6_ns_500_700    1             1                   361518
5_ns_300_500    1             1                   439327
                0             1           

In [17]:
selected_data = pd.DataFrame()

In [18]:
# for regular in  list(data_consider["REGULAR_FLAG"].unique()):
#     for consistent_flag in list(data_consider["CONSISTENT_FLAG"].unique()):
#         for buckets in list(data_consider["BUCKETS"].unique()):
#             print(regular , consistent_flag , buckets)
            
#             data_sel = data_consider[(data_consider["REGULAR_FLAG"] == regular) & (data_consider["CONSISTENT_FLAG"] == consistent_flag ) & (data_consider["BUCKETS"] == buckets)]
            
#             print(data_sel.shape)
for combo in list(data_consider["COMBO"].unique()):
    data_sel = data_consider[data_consider["COMBO"] == combo]
    X_train, Test = train_test_split(data_sel , test_size= 0.7 ,  stratify = data_sel['CONCAT'])
    Test["Flag"] = "Test"
    X_train["Flag"] = "Control" 


    temp_df = pd.concat([Test[["CUSTOMER_ID" , 'Flag']] ,  X_train[["CUSTOMER_ID" , 'Flag']]  ])   
    selected_data = pd.concat([selected_data, temp_df])


In [19]:
selected_data.groupby("Flag")["CUSTOMER_ID"].count()

Flag
Control    2228796
Test       5200556
Name: CUSTOMER_ID, dtype: int64

In [73]:
selected_data.shape

(7429352, 2)

In [20]:
merge_data_1 = data_consider.merge(selected_data , on = "CUSTOMER_ID" , how = 'left')

In [69]:
final_data = pd.DataFrame()
for column_val in ['AS_FLAG','SW_FLAG','PET_FLAG', 'MC2_FLAG']:
    pivot = merge_data_1.pivot_table(index=['BUCKETS' , 'REGULAR_FLAG' , 'CONSISTENT_FLAG',  column_val],
                        columns=['Flag'],
                        values = ["CUSTOMER_ID"],
                         aggfunc = 'count'
                        )
    pivot.columns =[str(s2)  +"_"+ str(s1) for (s1,s2) in pivot.columns.tolist()]
    pivot.reset_index(inplace=True)
    pivot["CHARACTERISTIC"] = column_val
    pivot.rename(columns = {column_val: "VALUE"} , inplace = True)
   
    final_data  = pd.concat([pivot, final_data], ignore_index=True)

final_data.rename(columns =   {"Control_CUSTOMER_ID": "Control" , "Test_CUSTOMER_ID": "Test" }, inplace = True)  
final_data = final_data[["BUCKETS" , "REGULAR_FLAG" , "CONSISTENT_FLAG", "CHARACTERISTIC" , "VALUE" , "Control" , "Test"]]


In [70]:
pivot_total = merge_data_1.pivot_table(index=['BUCKETS' , 'REGULAR_FLAG' , 'CONSISTENT_FLAG'],
                        columns=['Flag'],
                        values = ["CUSTOMER_ID"],
                        aggfunc = 'count'
                        )
pivot_total.columns =[str(s2)  +"_"+ str(s1) for (s1,s2) in pivot_total.columns.tolist()]
pivot_total.reset_index(inplace=True)
pivot_total.rename(columns =   {"Control_CUSTOMER_ID": "Control_Total" , "Test_CUSTOMER_ID": "Test_Total" }, inplace = True)

final_data = final_data.merge(pivot_total , on = ["BUCKETS" , "REGULAR_FLAG" , "CONSISTENT_FLAG"] , how = 'left') 
final_data.sort_values(by = ["BUCKETS" , "REGULAR_FLAG" , "CONSISTENT_FLAG"] , inplace = True)


In [72]:
final_data.to_csv("distribution.csv")

In [75]:
merge_data_1[["BUCKETS" , "REGULAR_FLAG" , "CONSISTENT_FLAG" , "CUSTOMER_ID" ,"Flag" ]].to_csv("new_cohort.csv")

In [50]:
test  = merge_data_1.groupby(['BUCKETS' , 'REGULAR_FLAG' , 'CONSISTENT_FLAG', 'Flag']).agg({"CUSTOMER_ID" : 'count' }).reset_index()
# test["AS_PERC"] = test["AS_FLAG"] * 100 / test["CUSTOMER_ID"]
# test["MC2_PERC"] = test["MC2_FLAG"] * 100 / test["CUSTOMER_ID"]

pivot

,BUCKETS,REGULAR_FLAG,CONSISTENT_FLAG,Control_CUSTOMER_ID,Test_CUSTOMER_ID
0,1_ns_1_50,0,0,217509,507521
1,1_ns_1_50,0,1,175954,410561
2,2_ns_51_150,0,0,273889,639076
3,2_ns_51_150,0,1,396033,924078
4,2_ns_51_150,1,1,16710,38991
5,3_ns_150_250,0,0,82161,191711
6,3_ns_150_250,0,1,190127,443631
7,3_ns_150_250,1,1,46345,108139
8,4_ns_250_300,0,0,23999,56000
9,4_ns_250_300,0,1,56078,130849


In [16]:
test.groupby(['BUCKETS' , 'REGULAR_FLAG' , 'CONSISTENT_FLAG' , 'Flag' ])['CUSTOMER_ID'].sum().sort_values()

BUCKETS         REGULAR_FLAG  CONSISTENT_FLAG  Flag   
8_ns_1100_1200  0             1                Control      3090
9_ns_1200_1500  0             1                Control      5511
8_ns_1100_1200  0             1                Test         7211
6_ns_500_700    0             0                Control     10078
9_ns_1200_1500  0             1                Test        12861
2_ns_51_150     1             1                Control     16710
8_ns_1100_1200  1             1                Control     22211
6_ns_500_700    0             0                Test        23517
4_ns_250_300    0             0                Control     23999
                1             1                Control     30477
5_ns_300_500    0             0                Control     30817
7_ns_700_1100   0             1                Control     34281
2_ns_51_150     1             1                Test        38991
3_ns_150_250    1             1                Control     46345
9_ns_1200_1500  1             1    

In [19]:
test

,BUCKETS,REGULAR_FLAG,CONSISTENT_FLAG,Flag,CUSTOMER_ID,MC2_FLAG,AS_FLAG,AS_PERC,MC2_PERC
0,1_ns_1_50,0,0,Control,217509,11865,90440,41.579889,5.454947
1,1_ns_1_50,0,0,Test,507521,27680,211026,41.579757,5.453962
2,1_ns_1_50,0,1,Control,175954,122100,80509,45.755709,69.393137
3,1_ns_1_50,0,1,Test,410561,284901,187853,45.755198,69.393099
4,2_ns_51_150,0,0,Control,273889,18101,122929,44.882781,6.608882
5,2_ns_51_150,0,0,Test,639076,42238,286835,44.882768,6.609230
6,2_ns_51_150,0,1,Control,396033,217392,207288,52.341093,54.892395
7,2_ns_51_150,0,1,Test,924078,507250,483673,52.341144,54.892552
8,2_ns_51_150,1,1,Control,16710,6125,8019,47.989228,36.654698
9,2_ns_51_150,1,1,Test,38991,14293,18716,48.000821,36.657177
